In [1]:
import os
import pandas as pd

In [2]:
df = pd.read_csv("page-blocks0.csv")

In [3]:
# 특징과 라벨 분리
X = df.drop('Class', axis = 1)
Y = df['Class']

In [4]:
# 학습 데이터와 평가 데이터 분할
from sklearn.model_selection import train_test_split
Train_X, Test_X, Train_Y, Test_Y = train_test_split(X, Y)

In [5]:
# 클래스 불균형 확인
Train_Y.value_counts()

negative    3676
positive     428
Name: Class, dtype: int64

In [6]:
Train_Y.replace({"negative":-1, "positive":1}, inplace = True)
Test_Y.replace({"negative":-1, "positive":1}, inplace = True)

In [7]:
# 클래스 불균형 비율 계산
Train_Y.value_counts().iloc[0] / Train_Y.value_counts().iloc[-1]

8.588785046728972

In [8]:
# kNN을 사용한 클래스 불균형 테스트
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.metrics import *
kNN_model = KNN(n_neighbors = 11).fit(Train_X, Train_Y)
pred_Y = kNN_model.predict(Test_X)
print(recall_score(Test_Y, pred_Y))
print(accuracy_score(Test_Y, pred_Y))

# 재현율이 60%로 불균형이 심각한 수준은 아니라고 보임

0.6412213740458015
0.956140350877193


In [9]:
# 불균형한 데이터 셋에서 높은 비율을 차지하던 클래스의 데이터 수를 줄인다.
# 데이터양이 감소하여 오히려 전체적인 성능이 감소할 수 있다.

from imblearn.under_sampling import NearMiss
# version = 2: 모든 소수 클래스 샘플까지의 평균 거리를 활용
NM_model = NearMiss(version = 2) 

# NearMiss 적용
u_Train_X, u_Train_Y = NM_model.fit_sample(Train_X, Train_Y)
u_Train_X = pd.DataFrame(u_Train_X, columns = X.columns)
u_Train_Y = pd.Series(u_Train_Y)

In [10]:
u_Train_Y.value_counts()

-1    428
 1    428
Name: Class, dtype: int64

In [11]:
# kNN 재적용을 통한 성능 변화 확인
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.metrics import *
kNN_model = KNN(n_neighbors = 11).fit(u_Train_X, u_Train_Y)
pred_Y = kNN_model.predict(Test_X)
print(recall_score(Test_Y, pred_Y))
print(accuracy_score(Test_Y, pred_Y))

# 재현율은 크게 올랐으나, 정확도가 크게 떨어짐 => 적당한 비율에 맞게 설정해야 함

0.9236641221374046
0.21637426900584794


In [12]:
from imblearn.under_sampling import NearMiss
# 5:1 정도의 비율로 언더샘플링 재수행

NM_model = NearMiss(version = 2, sampling_strategy = {1:u_Train_Y.value_counts().iloc[-1],
                                                      -1:u_Train_Y.value_counts().iloc[-1] * 5}) 
u_Train_X, u_Train_Y = NM_model.fit_sample(Train_X, Train_Y)
u_Train_X = pd.DataFrame(u_Train_X, columns = X.columns)
u_Train_Y = pd.Series(u_Train_Y)

In [13]:
u_Train_Y.value_counts()

-1    2140
 1     428
Name: Class, dtype: int64

In [14]:
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.metrics import *
kNN_model = KNN(n_neighbors = 11).fit(u_Train_X, u_Train_Y)
pred_Y = kNN_model.predict(Test_X)
print(recall_score(Test_Y, pred_Y))
print(accuracy_score(Test_Y, pred_Y))

0.8091603053435115
0.685672514619883
